Code

In [1]:
!nvidia-smi

Sat Aug 12 15:03:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 528.92       Driver Version: 528.92       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:02:00.0 Off |                  N/A |
| N/A   44C    P0    N/A /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# !pip3 install -U scipy

# !git clone https://github.com/neonbjb/tortoise-tts.git
%cd tortoise-tts

# !python3 setup.py install


d:\junk\web projects\v\tortoise-tts


In [4]:
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

import os

In [5]:
tts = TextToSpeech()

In [6]:
import os
import shutil

# Path to the folder containing the WAV files
folder_path = 'D:\\junk\\web projects\\v\\wav\\wav'

# List all files in the folder
file_list = os.listdir(folder_path)

# Filter only WAV files
wav_files = [file for file in file_list if file.lower().endswith('.wav')]

# Sort the WAV files
wav_files.sort()

# Rename and move the files
for index, old_filename in enumerate(wav_files, start=1):
    new_filename = f"{index}.wav"
    old_filepath = os.path.join(folder_path, old_filename)
    new_filepath = os.path.join(folder_path, new_filename)
    shutil.move(old_filepath, new_filepath)

print("WAV files renamed successfully.")


WAV files renamed successfully.


TEST 1 LINE TTS

In [7]:
# Define your own voice folder
VOICE_NAME='daniel'
text='Tortoise is a significant advancement in the field of natural language processing.'
preset = "fast"

# Generate with your own voice
voice_samples, conditioning_latents = load_voice(VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated-{VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{VOICE_NAME}.wav')

Generating autoregressive samples..


  0%|          | 0/96 [00:00<?, ?it/s]

SRT TO TEXT

In [ ]:
def combine_srt_text(srt_file_path):
    with open(srt_file_path, 'r') as file:
        lines = file.readlines()

    text = ''
    current_text = []

    for line in lines:
        line = line.strip()
        if line.isdigit():
            if current_text:
                text += ' '.join(current_text) + ' '
                current_text = []
        elif line and not line.startswith(('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')):
            current_text.append(line)

    if current_text:
        text += ' '.join(current_text)

    return text

srt_file_path = 'test.srt'
combined_text = combine_srt_text(srt_file_path)

with open('test.txt', 'w') as output_file:
    output_file.write(combined_text)


READ TEXT FILE AND TTS

In [ ]:
# Define your own voice folder
VOICE_NAME = 'daniel'
preset = "fast"

# preset values are: "fast", "ultra-fast"
# Load voice samples and conditioning latents
voice_samples, conditioning_latents = load_voice(VOICE_NAME)

# Read text from a text file sentence by sentence
input_text_file = 'test.txt'
output_waveform = []

with open(input_text_file, 'r') as f:
    text = f.read()
    sentences = text.split('. ')  # Assuming sentences are separated by ". "
    for sentence in sentences:
        sentence = sentence.strip()

        # Generate speech for the current sentence
        gen = tts.tts_with_preset(sentence, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                                  preset=preset)

        # Append generated speech to the output waveform
        output_waveform.append(gen.squeeze(0).cpu())

# Combine generated waveforms into a single waveform
combined_waveform = torch.cat(output_waveform, dim=1)

# Create the "result" folder if it doesn't exist
output_folder = 'result'
os.makedirs(output_folder, exist_ok=True)

# Save the combined waveform as a WAV file in the "result" folder
combined_output_wav = os.path.join(output_folder, f'combined-generated-{VOICE_NAME}.wav')
torchaudio.save(combined_output_wav, combined_waveform, 24000)

# Display the combined WAV file
IPython.display.Audio(combined_output_wav)
